<a href="https://colab.research.google.com/github/erinkim16/dishdetective/blob/main/311_girlies_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas
!pip install word2number

import pandas as pd
import re
from word2number import w2n


  Preparing metadata (setup.py) ... done
  Created wheel for word2number: filename=word2number-1.1-py3-none-any.whl size=5568 sha256=fb77d84f55c88857ca5baccfe54084d2e56d4690173d364d60f7b515a03e2183
  Stored in directory: /root/.cache/pip/wheels/cd/ef/ae/073b491b14d25e2efafcffca9e16b2ee6d114ec5c643ba4f06
Successfully built word2number


In [32]:

# Load the CSV file into a DataFrame
df = pd.read_csv('/content/cleaned_data_combined_modified.csv')

# Display the first few rows of the DataFrame
df.head()



,id,"Q1: From a scale 1 to 5, how complex is it to make this food? (Where 1 is the most simple, and 5 is the most complex)",Q2: How many ingredients would you expect this food item to contain?,Q3: In what setting would you expect this food to be served? Please check all that apply,Q4: How much would you expect to pay for one serving of this food item?,Q5: What movie do you think of when thinking of this food item?,Q6: What drink would you pair with this food item?,"Q7: When you think about this food item, who does it remind you of?",Q8: How much hot sauce would you add to this food item?,Label
0,716549,3,6,"Week day lunch,At a party,Late night snack",5,Cloudy with a Chance of Meatballs,Coke,Friends,A little (mild),Pizza
1,715742,4,"bread, meet","Week day lunch,At a party,Late night snack",5$ for a large piece,All sort of american young boy movies,Coke,"Friends,Teachers,Strangers",NaN,Pizza
2,727333,3,5,"Week day lunch,Week day dinner,Weekend lunch,W...",10dollar,action movie,cola,Friends,A moderate amount (medium),Pizza
3,606874,4,6-7,"Week day lunch,Week day dinner,Weekend lunch,W...",$3,Mamma Mia,Soda,"Siblings,Friends,Teachers",I will have some of this food item with my hot...,Pizza
4,505318,2,3 or more,"Week day lunch,Week day dinner,Weekend lunch,W...",$5,Cloudy with a chance of meatballs,Soda,"Siblings,Friends",A little (mild),Pizza


How to parse "How many ingredients" Q2

*  take average
*  take first number
* take last number
* count how many spaces in between the words or commas



In [33]:
# Define a dictionary to map month abbreviations to numbers
month_map = {
    'Jan': '1',
    'Feb': '2',
    'Mar': '3',
    'Apr': '4',
    'May': '5',
    'Jun': '6',
    'Jul': '7',
    'Aug': '8',
    'Sep': '9',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

# Function to convert date format
def convert_date_format(date_str):
    if isinstance(date_str, str) and '-' in date_str:
        try:
            day, month_abbr = date_str.split('-')
            month_num = month_map.get(month_abbr, month_abbr)
            return f"{int(month_num)}-{int(day)}"
        except ValueError:
           return date_str
    return date_str

# Apply the function to the Q2 column
df['Q2: How many ingredients would you expect this food item to contain?'] = df['Q2: How many ingredients would you expect this food item to contain?'].apply(convert_date_format)

# Function to take the average if there's a range(either with '-' or 'to')
def average_of_range(text):
  if isinstance(text, str):
    range_match = re.search(r'(\d+)[\s-]+(\d+)', text)
    range_match2 = re.search(r'(\d+)\s* to \s*(\d+)', text)

    if range_match is not None:
      num1, num2 = int(range_match.group(1)), int(range_match.group(2))
      return int((num1 + num2) / 2 )# Take the average of the range
    elif range_match2 is not None:
      num1, num2 = int(range_match2.group(1)), int(range_match2.group(2))
      return int((num1 + num2) / 2)
    else:
      return text

# Apply the function to the Q2 column
df['Q2: How many ingredients would you expect this food item to contain?'] = df['Q2: How many ingredients would you expect this food item to contain?'].apply(average_of_range)


# Function to extract numerical values in data entry
def extract_number(text):
  # check if entry is string
  if isinstance(text, str):
    # search for any digits in the text
    # note, this is called after all previous functions, which should indicate high likelihood
    # that what's left is the answer value with only v/ small margin of error
    num = re.search(r'(\d+)', text)

    # if found, return as int
    if num is not None:
      return int(num.group(1))

    # if no match, return original entry
    else:
      return text
  # if not, return original entry
  else:
    return text

# Apply the function to the Q2 column
df['Q2: How many ingredients would you expect this food item to contain?'] = df['Q2: How many ingredients would you expect this food item to contain?'].apply(extract_number)


def convert_wrds(text):
    if isinstance(text, str):
        # Regex to find hyphenated numbers (e.g., "twenty-two", "one-hundred-three")
        hyphenated_pattern = r'\b(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion)(?:-\s?(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion))+\b'

        # Regex to find single (non-hyphenated) number words
        single_pattern = r'\b(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion)\b'

        # Look for a hyphenated number first
        hyphenated_match = re.search(hyphenated_pattern, text, re.IGNORECASE)
        if hyphenated_match:
            try:
                return w2n.word_to_num(hyphenated_match.group(0))  # Convert full hyphenated number
            except ValueError:
                return hyphenated_match.group(0)  # Return as text if conversion fails

        # If no hyphenated number, look for a single non-hyphenated number
        single_match = re.search(single_pattern, text, re.IGNORECASE)
        if single_match:
            try:
                return w2n.word_to_num(single_match.group(1))  # Convert first non-hyphenated number
            except ValueError:
                return single_match.group(1)  # Return as text if conversion fails

    return text  # Return original text if no numbers found

# Apply function to the Q2 column
df['Q2: How many ingredients would you expect this food item to contain?'] = df[
    'Q2: How many ingredients would you expect this food item to contain?'
].apply(convert_wrds)

# Function to count the number of ingredients if responses are comma-separated or use '*'
def count_ingredients(text):
    if isinstance(text, str):  # Ensure it's a string
        if ',' in text:  # Count comma-separated items
            return len([item.strip() for item in text.split(',') if item.strip()])
        elif '* ' in text:  # Count bullet-pointed items
            return len([item.strip() for item in text.split('* ') if item.strip()])
    return text  # If it's a single ingredient or doesn't match conditions, return 1

# Apply the function to the Q2 column
df['Q2: How many ingredients would you expect this food item to contain?'] = df[
    'Q2: How many ingredients would you expect this food item to contain?'
].apply(count_ingredients)


# Function to identify likely 'outliers' (ie. stories/anecdotes, 'i don't knows', etc.)
def outlier_rest(text):
  # check if entry is string
  if isinstance(text, str):
    # search for any text characters
    is_match = re.search(r'(\s+)', text)
    # if found, return 'None' as a string to identify an outlier in the data
    if is_match is not None:
      return 'None'

    # if not, return original entry
    else:
      return text
  # if not, return original
  else:
    return text


How to parse "How much would you pay" Q4
*   first number seen
*   average of many numbers


In [34]:
# Function to take the average if there's a range(either with '-' or 'to')
def q4_range(text):
  # make sure data entry is a string
  if isinstance(text, str):

    # case where there's a range and no dollar sign (eg. 2-5)
    no_sign = re.search(r'(\d+)\s*(?:-|to|up to)\s*(\d+)', text)

    # case where there's a range w/ a dollar sign at the front (eg. $2-$5)
    sign_front = re.search(r'(\d+)\s*(?:-|to|up to)\s*\$(\d+)', text)

    # case where there's a range w/ a dollar sign at the back (eg. 2$-5$)
    sign_back = re.search(r'(\d+)\$\s*(?:-|to|up to)\s*(\d+)', text)

    # if one is not None, ie a match was found, extract #s and take average
    if no_sign is not None:
      num1, num2 = int(no_sign.group(1)), int(no_sign.group(2))
      return (num1 + num2) / 2

    elif sign_front is not None:
      num1, num2 = int(sign_front.group(1)), int(sign_front.group(2))
      return (num1 + num2) / 2

    elif sign_back is not None:
      num1, num2 = int(sign_back.group(1)), int(sign_back.group(2))
      return (num1 + num2) / 2

    # if no match found, return original data entry
    else:
      return text
  # if not string, return original entry
  else:
    return text

# Function to find float money values and return as floats
def floats(text):
  # check if data entry is a string
  if isinstance(text, str):
    # find values in full 'monetary' form(ie. 12.00 instead of 12, or 12.99)
    float_vars = re.search(r'(\d+\.\d{2})', text)

    # if not None, ie match found, return the value in float format
    if float_vars is not None:
      return float(float_vars.group(1))

    # if no match found, return original entry
    else:
      return text
  # if not string, return original entry
  else:
    return text

# Function to find values with written 'money' indications (ie. 12 dollars)
def dollars(text):
  # check if entry is a string
  if isinstance(text, str):
    # search for variations of 'money-indicative' words that have a number directly preceeding it
    dollar_vars = re.search(r'(\d+)\s*(dollars|dollar|dollor|bucks|buck)', text, re.IGNORECASE)
    currencies = re.search(r'(\d+)\s*(CAD|USD|Canadian)', text, re.IGNORECASE)

    # if match for either found, return float value of that number
    if dollar_vars is not None:
      return float(dollar_vars.group(1))
    elif currencies is not None:
      return float(currencies.group(1))

    # if no match found, return original
    else:
      return text
  # if not, return original entry
  else:
    return text

# Function to find values directly preceding or following a '$' symbol
def dollar_sign(text):
  # make sure entry is a string
  if isinstance(text, str):

    # search for numbers either before or after '$'
    # note, this is called after q4_range to not misselect $4-$5 as just 4
    sign_first = re.search(r'\$(\d+)', text)
    sign_after = re.search(r'(\d+)\s*\$', text)

    # if found, return value as float
    if sign_first is not None:
      return float(sign_first.group(1))
    elif sign_after is not None:
      return float(sign_after.group(1))

    # if no match found, return original entry
    else:
      return text
  # if not, return original entry
  else:
    return text

# Function to extract numerical values in data entry
def extract_number(text):
  # check if entry is string
  if isinstance(text, str):
    # search for any digits in the text
    # note, this is called after all previous functions, which should indicate high likelihood
    # that what's left is the answer value with only v/ small margin of error
    num = re.search(r'(\d+)', text)

    # if found, return as float
    if num is not None:
      return float(num.group(1))

    # if no match, return original entry
    else:
      return text
  # if not, return original entry
  else:
    return text

# Function to find written numbercal values and convert into proper form

def convert_wrds(text):
    if isinstance(text, str):
        # Regex to find hyphenated numbers (e.g., "twenty-two", "one-hundred-three")
        hyphenated_pattern = r'\b(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion)(?:-\s?(?:one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion))+\b'

        # Regex to find single (non-hyphenated) number words
        single_pattern = r'\b(one|two|three|four|five|six|seven|eight|nine|ten|eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand|million|billion)\b'

        # Look for a hyphenated number first
        hyphenated_match = re.search(hyphenated_pattern, text, re.IGNORECASE)
        if hyphenated_match:
            try:
                return w2n.word_to_num(hyphenated_match.group(0))  # Convert full hyphenated number
            except ValueError:
                return hyphenated_match.group(0)  # Return as text if conversion fails

        # If no hyphenated number, look for a single non-hyphenated number
        single_match = re.search(single_pattern, text, re.IGNORECASE)
        if single_match:
            try:
                return w2n.word_to_num(single_match.group(1))  # Convert first non-hyphenated number
            except ValueError:
                return single_match.group(1)  # Return as text if conversion fails

    return text  # Return original text if no numbers found

# Apply function to the Q2 column
df['Q2: How many ingredients would you expect this food item to contain?'] = df[
    'Q2: How many ingredients would you expect this food item to contain?'
].apply(convert_wrds)

# Function to identify likely 'outliers' (ie. stories/anecdotes, 'i don't knows', etc.)
def outlier_rest(text):
  # check if entry is string
  if isinstance(text, str):
    # search for any text characters
    is_match = re.search(r'(\s+)', text)
    # if found, return 'None' as a string to identify an outlier in the data
    if is_match is not None:
      return 'None'

    # if not, return original entry
    else:
      return text
  # if not, return original
  else:
    return text

# Applying all functions to the Q4 column
# Important note! ORDER MATTERS! Specific order to identify cases, and not *mis*-identify cases with only very small margin of error

# Apply the date function to the Q4 column
df['Q4: How much would you expect to pay for one serving of this food item?'] = df['Q4: How much would you expect to pay for one serving of this food item?'].apply(convert_date_format)

# Apply the q4_range function to the Q4 column
df['Q4: How much would you expect to pay for one serving of this food item?'] = df['Q4: How much would you expect to pay for one serving of this food item?'].apply(q4_range)

# Apply the floats function to the Q4 column
df['Q4: How much would you expect to pay for one serving of this food item?'] = df['Q4: How much would you expect to pay for one serving of this food item?'].apply(floats)

# Apply the dollars function to the Q4 column
df['Q4: How much would you expect to pay for one serving of this food item?'] = df['Q4: How much would you expect to pay for one serving of this food item?'].apply(dollars)

# Apply the dollar_sign function to the Q4 column
df['Q4: How much would you expect to pay for one serving of this food item?'] = df['Q4: How much would you expect to pay for one serving of this food item?'].apply(dollar_sign)

# Apply the extract number function to the Q4 column
df['Q4: How much would you expect to pay for one serving of this food item?'] = df['Q4: How much would you expect to pay for one serving of this food item?'].apply(extract_number)

# Apply the convert words function to the Q4 column
df['Q4: How much would you expect to pay for one serving of this food item?'] = df['Q4: How much would you expect to pay for one serving of this food item?'].apply(convert_wrds)

# Apply the outlier_rest function to the Q4 column
df['Q4: How much would you expect to pay for one serving of this food item?'] = df['Q4: How much would you expect to pay for one serving of this food item?'].apply(outlier_rest)

df.to_csv('\content\test.csv', index=False)

Q6: can parse the data by classifying similar ones, like coke, soda, sprite, wine, ...


*   by whcihever one we find first from sort data set alphabetically
* find all matches from data set and randomly choose it
* find all matches from data set and pick which one comes earliest (first)
*   by whichever one we find first from sort data set by frequency of dataset.






In [ ]:
# pandas, numpy, download stuff

######### drinks ############

# lowercase everything

# find the median word length to decide how to make split
# of yap vs actual movie name / drink name

# ex: only 2 words length is enough to describe most drinks, only use that as
# "data set " comparaison

# group together based on persoal thought n how generalizable it is
# using a dataset , soda~softdrink~pop same,

# sort "data set" popularity (counts)

# for long answers for drinks, to match with the "data set"
# and take first match

# keep track of any data that couldn't be matched
# will see if we can label it as 'None' / noise
# or if we need to include more drinks in our 'dataset'

######## movie #############




Q5 Movie


In [ ]:
# remove everyting after :......
# if there's a number , like shrek 2, remove the number 2, but if its 16 candles,
# don't remove cos its likely part of the title

# lowercase thing

# divide based on median, so we divide the yapping rants vs actual titles (assuming its likely chances)

# research to try and find data set with all movies ever

# For those that couldn't find a movie and was too big, first try to find the first actual movie name
# that shows up, otherwise label it as 'None'